In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
import random
import matplotlib
from matplotlib import pyplot as plt
import statistics
from pandas import DataFrame
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential,Input,Model
from tensorflow.python.keras.layers import Dense, Dropout, Reshape, Flatten, Activation
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Concatenate
#from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam, RMSprop
#from keras import backend as K
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, Callback
from tensorflow.compat.v1.keras.backend import set_session
from os.path import basename
#import seaborn as sns
from tensorflow.python.keras.models import load_model 
from tensorflow.python.keras.utils.vis_utils import plot_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from scipy.stats import pearsonr

In [ ]:
tf.__version__

In [ ]:
'server'

## GPU check

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# for i in range(len(gpus)):
#     tf.config.experimental.set_memory_growth(gpus[i], True)

In [ ]:
os.getcwd()

## Load input file

In [ ]:
x_test = np.load('./path/test_x_file') # Enter the path to your file, file shape must be in the form (n, 512, 512, 3)
y_test = np.load('./path/test_y_file') # Enter the path to your file, file shape must be in the form (n, )

In [ ]:
with tf.device('/GPU:0'):
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

In [ ]:
batch_size = 32 # Assign your desired batch size to the batch_size variable. Default is 32
with tf.device('/GPU:0'): # If you have a GPU and more than one, enter the GPU number to use 
    test_data = tf.data.Dataset.from_tensor_slices(x_test)
    test_data = test_data.batch(batch_size) 
    test_data = test_data.with_options(options)

## Load model and prediction

In [ ]:
os.getcwd()

In [ ]:
model = load_model('./path/characteristics_model.h5')
# Set the path to the model with the desired characteristics
# If you want the saved model, contact us

In [ ]:
predict = model.predict(test_data, verbose = 1)

In [ ]:
predict.shape, y_test.shape

In [ ]:
predict

In [ ]:
y_test

In [ ]:
predict_test_reshape = np.reshape(predict,('row_in_your_data',)) 

In [ ]:
predict_test_reshape.shape

In [ ]:
predict_test_reshape

In [ ]:
predict_test_reshape = np.round(predict_test_reshape,0)

In [ ]:
predict_train_reshape, predict_valid_reshape, predict_test_reshape

In [ ]:
predict_test_reshape = predict_test_reshape.astype('uint8')

In [ ]:
predict_test_reshape

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, plot_roc_curve, roc_auc_score

In [ ]:
target_names = ["Class {}".format(i) for i in range(2)]
print(classification_report(y_test, predict_test_reshape, target_names=target_names))

In [ ]:
confusion_matrix(y_test, predict_test_reshape, labels=[0,1])

In [ ]:
y_test_reshape = np.reshape(y_test,('row_in_your_data',1))

In [ ]:

# Compute ROC curve and ROC area for each class
test_fpr = dict()
test_tpr = dict()
test_roc_auc = dict()
test_ll = dict()

for i in range(2):
    test_fpr[i], test_tpr[i], test_ll[i] = roc_curve(y_test, predict_test_reshape)
    test_roc_auc[i] = auc(test_fpr[i], test_tpr[i])

    # Compute micro-average ROC curve and ROC area
test_fpr["micro"], test_tpr["micro"], _ = roc_curve(y_test, predict_test_reshape)
test_roc_auc["micro"] = auc(test_fpr["micro"], test_tpr["micro"])

In [ ]:
def plot_roc_curve(l_fper, l_tper, ):
    plt.plot(l_fper,l_tper , color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()

In [ ]:
predict_test_prob_reshape = np.reshape(predict, ('row_in_your_data',1))

In [ ]:
test_auc_score = roc_auc_score(y_test, predict_test_prob_reshape)
test_auc_score

In [ ]:
test_fper, test_tper, test_thresholds = roc_curve(y_test, predict_test_prob_reshape)

In [ ]:
plot_roc_curve(test_fper, test_tper)